In [ ]:
!pip install git+https://github.com/Abdulrasheed1729/polar-semeval-2026.git --quie

In [ ]:
import pandas as pd

from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np
from polar_semeval.semeval_dataset import SemEvalDataset, LanguageType

import torch

from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    RobertaForSequenceClassification
)
from torch.utils.data import Dataset

In [ ]:
from sklearn.model_selection import train_test_split

dataset = SemEvalDataset(subtask=2, lang_key=LanguageType.ENG)

train, val = train_test_split(dataset.dataframe, test_size=0.2, random_state=42)

In [ ]:
class PolarizationDataset(torch.utils.data.Dataset):
  def __init__(self,texts,labels,tokenizer,max_length =128, dtype = torch.long):
    self.texts=texts
    self.labels=labels
    self.tokenizer= tokenizer
    self.max_length = max_length # Store max_length
    self.dtype = dtype

  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text=self.texts[idx]
    label=self.labels[idx]
    encoding=self.tokenizer(text,truncation=True,padding=False,max_length=self.max_length,return_tensors='pt')

    # Ensure consistent tensor conversion for all items
    item = {key: encoding[key].squeeze() for key in encoding.keys()}
    item['labels'] = torch.tensor(label, dtype=self.dtype)
    return item

## Model Training

In [ ]:
# Load the tokenizer
MODEL_NAME = 'distilbert/distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
